In [1]:
                                                                        # Programmer: Vishal R. Gangaram
# imports
import quandl
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from scipy import stats
from scipy.optimize import minimize

%matplotlib inline
quandl.ApiConfig.api_key = ""


from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
start = pd.to_datetime('2016-11-03')
end = pd.to_datetime('2020-11-10')
api_key = ''
#These are real time data sets
#AAPL_data = quandl.get("EOD/AAPL", authtoken=api_key,start_date=start, end_date=end)

In [3]:
class PortfolioOptimizer: # Class with the capability of constructing a user portfolio and optimizing it
    
    def __init__(self, dataframes=[], stock_names=[], return_df=[], close_df=[]):
        self.dataframes = dataframes # inputted dataframes from quandl
        self.stock_names = stock_names # names of the stock attributed to each dataframe in self.dataframes
        self.return_df = return_df # list of return arrays from each dataframe in self.dataframes
        self.close_df = close_df # list of close array from each dataframe in self.dataframe
        
    def add_stock(self, ticker): # Add stock method: adds stock to self.dataframe, pulls from quandl
        self.stock_names.append(ticker)
        ticker = quandl.get("eod/{stock}".format(stock = ticker), authtoken=api_key,start_date=start, end_date=end)
        ticker = ticker.reset_index()
        self.dataframes.append(ticker)
    
    def add_holdings(self, holdings): # Adds the user's current position of each asset in within self.dataframes
        self.holdings = holdings # list that contains the user's current position in each stock within self.dataframes        

    def construct_df(self, column): # Method that constructs a dataframe by concatinating specific arrays 
                                        # that were pulled from each dataframe in self.dataframes
        column_lst = []
        for i in range(len(self.dataframes)): # for loop that looks though each dataframe for the specified array
            column_lst.append(self.dataframes[i][str(column)]) # appends array
        stock_columns = pd.concat(column_lst, axis=1) # concatinates array
        
        if column == 'returns': # if the constructed df is a 'return' dataframe
            self.return_df.append(stock_columns) # append dataframe to self.return_df attribute
        elif column =='Close': # if the consrtucted df is a 'Close' dataframe
                self.close_df.append(stock_columns) # append dataframe to self.close_df attribute
        else: pass
        stock_columns = stock_columns.reset_index()
        return stock_columns # returns constructed dataframe
    
    def get_returns(self): # function that computes returns of a given stock   
        for i in range(len(self.dataframes)): # math involved in computing returns
            self.dataframes[i]['returns'] = self.dataframes[i]['Close'].pct_change(1)
        # doesn't return anything, instead it appends the return column to the already existing pandas df

    def get_log_returns(self): # function that computes logarithmic returns of a given stock
        log_ret = np.log(self.close_df[0] / self.close_df[0].shift(1))
        return log_ret #returns logarithmic return
    
    def calculate_ratio(self, days, ratioType): #calculates portfolio sharpe or sortino ratio of asset r_i
        spy_data = quandl.get("eod/spy", authtoken=api_key, start_date= start, end_date=end) # pings quandl
        close_df = self.close_df[0] # assigns close_df to be first index of close_df (a dataframe within an array)
        log_ret = np.log(self.close_df[0] / self.close_df[0].shift(1))
        str(ratioType).lower() # catches bad inputs, ensures if/else statement works properly
        exp_ret = np.sum((log_ret.mean() * days)) # calculates expected return 
        
        if ratioType == 'sharpe':
            exp_vol = log_ret.std() * np.sqrt(days) # calculates sharpe ratio
            sharpe_ratio = exp_ret / exp_vol
            print('The Sharpe Ratio is: {sharpe} '.format(sharpe=sharpe_ratio))
            self.sharpe_ratio = sharpe_ratio # prints sharpe ratio, assigns sharpe ratio to self.sharpe_ratio
        
        elif ratioType == 'sortino':
            exp_vol = log_ret[log_ret < 0].std() * np.sqrt(days) # same calculation, except only with downward deviation now
            sortino_ratio = exp_ret / exp_vol
            print('The Sortino Ratio is: {sortino} '.format(sortino=sortino_ratio)) # prints sortio ratio
            self.sortino_ratio = sortino_ratio
            
        else:
            pass
        
    def calculate_weights(self): # calculates the weights of portfolio from input array
        self.weights_array = self.holdings/np.sum(self.holdings) # caluclates weights
        #self.weights_array.append(i for i in weights_array)
        return self.weights_array

    def calculate_portfolio_sharpe(self, weights_array, returns_array, days): #calculates sharpe ratio of entire portfolio r_p
        exp_ret = np.sum( (returns_array.mean()  * days)) # Expected return
        exp_vol = returns_array.std() * np.sqrt(days) # Expected volatility
        sharpe_ratio = exp_ret/exp_vol
        print('The Sharpe Ratio is: ')
        return sharpe_ratio # returns sharpe ratio
    
    def get_ret_vol_sr(self):
        log_ret = self.get_log_returns() # gets logarithimic returns
        ret = np.sum(log_ret.mean() * self.weights_array) * 252 # gets returns
        vol = np.sqrt(np.dot(self.weights_array.T, np.dot(log_ret.cov()*252,self.weights_array))) # gets volatility
        sr=ret/vol # sharpe ratio
        return np.array([ret,vol,sr])
    
    def plot_column(self, index, column):
        
        if str(index).lower() == 'all': # checks if user wants to plot all stocks in portfolio
            fig = go.Figure() # creates figure object
            fig_lst = self.construct_df(column) # creates dataframe depending on what the user asked for
            
            for i in range(len(self.dataframes)):
                fig.add_trace(go.Scatter(
                x=self.dataframes[0]['Date'],
                y=self.dataframes[i][str(column)],
                name=str(self.stock_names[i])))    # this sets its legend entry
                              
            fig.update_layout(title="All " + str(column),
                              xaxis_title="Time",
                              yaxis_title="Asset Value",
                              legend_title="Stocks",
                              font=dict(family="Courier New, monospace",
                                        size=18,
                                        color="RebeccaPurple"))
            fig.show()   
        else: # plots self.dataframe[index][str(column)]
            
            fig = go.Figure(data=[go.Scatter(x=self.dataframes[index]['Date'], 
                                             y=self.dataframes[index][str(column)])],
                                             layout=go.Layout(title=go.layout.Title(
                                                              text=str(self.stock_names[index]+ ' ' + str(column)))))
            fig.update_layout(
                              xaxis_title="Time",
                              yaxis_title="Asset Value",
                              font=dict(family="Courier New, monospace",
                                        size=18,
                                        color="RebeccaPurple"))
            fig.show()
        
    #def plot_column_candlestick(self, index, column):
        
    def neg_sharpe(self): return self.get_ret_vol_sr()[2] * -1
    def check_sum(self): return np.sum(self.weights_array) - 1   # return 0 if the sum of the weights is 1
    def minimize_volatility(self): return self.get_ret_vol_sr()
    
    ## These two functions are currently broken...
    def optimizer(self): # optimizes portfolio, returns list of floats - eventually will apply floats to self.weights_array
        #def neg_sharpe(self): return self.get_ret_vol_sr()[2] * -1
        cons = ({'type':'eq','fun': self.check_sum()}) # constraints
        bounds = ((0,1),(0,1),(0,1),(0,1)) # bounds
        init_guess = [0.25,0.25,0.25,0.25] # initial guess 
        #neg_sharpe = self.neg_sharpe()
        opt_results = minimize(self.neg_sharpe, init_guess,method='SLSQP',bounds=bounds,constraints=cons) # scipy minimize function
        return opt_results # optimized via least squares method
        
    def get_frontier_volatility(self): #function that iteratively computes curve on volatility frontier
        def neg_sharpe(self): return self.get_ret_vol_sr()[2] * -1
        init_guess = [0.25,0.25,0.25,0.25] # initial guess
        frontier_volatility = [] # create list variable
        for possible_return in frontier_y: # generates list of optimized points on efficient frontier of Markowitz curve
            cons = ({'type':'eq','fun':check_sum()}, # checksum == float, broken here
                {'type':'eq','fun':lambda w: self.get_ret_vol_sr(self.weights_array)[0]-possible_return})
            result = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons) #self.minimize_volatility == float, broken here as well...
            frontier_volatility.append(result['fun'])
        return frontier_volatility # returns list of optimized curve values
    
    

## Testing

In [4]:
RyanPortfolio = PortfolioOptimizer() # creates instance of PortfolioOptmizer object
holdings = [1,5,7,3,9] # arbitrary holdings array
RyanPortfolio.add_holdings(holdings) # calling add_holdings() method
RyanPortfolio.calculate_weights() # calling calculate_weights() method, => that both add_holdings()
                                # and calculate_weights() work properly due to dependencies

array([0.04, 0.2 , 0.28, 0.12, 0.36])

In [5]:
# calling add_stock() method, updates PortfolioOptimizer object
RyanPortfolio.add_stock('TSLA')
RyanPortfolio.add_stock('AAPL')
RyanPortfolio.add_stock('GOOGL')
RyanPortfolio.add_stock('DIS')
RyanPortfolio.add_stock("SPY")

RyanPortfolio.stock_names # stock_names attribute updated => add_stock() method works

['TSLA', 'AAPL', 'GOOGL', 'DIS', 'SPY']

In [6]:
# calling get_returns(), returns nothing
RyanPortfolio.get_returns() 

# 'returns' array properly concatinated to original df => get_return() method works
RyanPortfolio.dataframes[0]['returns'] 

0            NaN
1       0.016754
2       0.013906
3       0.008954
4      -0.025033
          ...   
1007   -0.006888
1008    0.040643
1009   -0.018581
1010   -0.020212
1011   -0.025875
Name: returns, Length: 1012, dtype: float64

In [7]:
# calling construct_df() method:
RyanPortfolio.construct_df('returns') # creates 'returns' dataframe
RyanPortfolio.construct_df('Close') # creates 'Close' dataframe

# close_df attribute updated => construct_df() method works 
RyanPortfolio.close_df[0] 

,Close,Close,Close,Close,Close
0,187.42,109.83,782.19,93.37,208.779999
1,190.56,108.84,781.10,92.45,208.550003
2,193.21,110.41,802.03,94.43,213.149994
3,194.94,111.06,811.98,94.38,214.110001
4,190.06,110.88,805.59,94.64,216.380005
...,...,...,...,...,...
1007,420.98,114.95,1745.85,125.07,343.540000
1008,438.09,119.03,1762.50,126.96,350.240000
1009,429.95,118.69,1759.73,127.46,350.160000
1010,421.26,116.32,1761.42,142.59,354.560000


In [8]:
# calling calculate_ratio() method:
RyanPortfolio.calculate_ratio(252, 'sharpe') # returns sharpe ratio of each stock in PortfolioOptimizer object

The Sharpe Ratio is: Close    0.671810
Close    0.864196
Close    2.319069
Close    2.248554
Close    3.190510
dtype: float64 


In [16]:
# Calling plot_column() function:
RyanPortfolio.plot_column(2,'Close') # checks if first condition works

In [10]:
RyanPortfolio.plot_column(0,'Close') #checks if second condition works